## SQL - SQLite3

### Packages

In [1]:
%%bash
echo $VIRTUAL_ENV
# PySQLite is a part of the Python Standard library

/home/grossnik/venv/bdl03-1-jpy-3.8


In [2]:
!pip3 list | grep sqlite3

In [3]:
import sqlite3
import pathlib
import os

### Configuration

In [4]:
p = pathlib.Path.home().joinpath('bdl03-1/sqlite')
p.mkdir(parents=True, exist_ok=True)
base_dir = str(p)
db_file = f"{base_dir}/pythonsqlite.db"
db_file

'/home/grossnik/bdl03-1/sqlite/pythonsqlite.db'

In [5]:
# delete the file
if os.path.exists(db_file):
  os.remove(db_file) 

! ls -la "${HOME}/bdl03-1/sqlite"

total 8
drwxrwxr-x 2 grossnik grossnik 4096 Feb 15 18:44 .
drwxrwxr-x 3 grossnik grossnik 4096 Feb 15 18:06 ..


### Create Database and Tables

In [6]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)

    return conn


In [7]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)


In [8]:
def create_tables(database):
    sql_create_projects_table = """
CREATE TABLE IF NOT EXISTS projects (
id         INTEGER PRIMARY KEY,
name       TEXT    NOT NULL   ,
begin_date TEXT               ,
end_date   TEXT
);
"""

    sql_create_tasks_table = """
CREATE TABLE IF NOT EXISTS tasks (
id          INTEGER PRIMARY KEY,
name        TEXT    NOT NULL   ,
priority    INTEGER            ,
status_id   INTEGER NOT NULL   ,
project_id  INTEGER NOT NULL   ,
begin_date  TEXT    NOT NULL   ,
end_date    TEXT    NOT NULL   ,
FOREIGN KEY (project_id) REFERENCES projects (id)
);
"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_projects_table)

        # create tasks table
        create_table(conn, sql_create_tasks_table)
    else:
        print("Error! cannot create the database connection.")


In [9]:
create_tables(db_file)

In [10]:
! ls -l "${HOME}/bdl03-1/sqlite"

total 12
-rw-r--r-- 1 grossnik grossnik 12288 Feb 15 18:44 pythonsqlite.db


### Insert some date

In [11]:
def create_project(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = '''
INSERT INTO projects(name,begin_date,end_date) VALUES(?,?,?)
'''
    cur = conn.cursor()
    cur.execute(sql, project)
    return cur.lastrowid


In [12]:
def create_task(conn, task):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = '''
INSERT INTO tasks(name,priority,status_id,project_id,begin_date,end_date)
  VALUES(?,?,?,?,?,?)
'''
    cur = conn.cursor()
    cur.execute(sql, task)
    return cur.lastrowid


In [13]:
database = db_file

# create a database connection
conn = create_connection(database)
with conn:
    # create a new project
    project = ('Cool App with SQLite & Python',
               '2015-01-01',
               '2015-01-30')
    project_id = create_project(conn, project)

    # tasks
    task_1 = ('Analyze the requirements of the app',
              1,
              1,
              project_id,
              '2015-01-01',
              '2015-01-02')

    task_2 = ('Confirm with user about the top requirements',
              1,
              1,
              project_id,
              '2015-01-03',
              '2015-01-05')

    # create tasks
    create_task(conn, task_1)
    create_task(conn, task_2)


### Select some data

In [14]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM tasks")

    rows = cur.fetchall()

    for row in rows:
        print(row)


In [15]:
def select_task_by_priority(conn, priority):
    """
    Query tasks by priority
    :param conn: the Connection object
    :param priority:
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM tasks WHERE priority=?", (priority,))

    rows = cur.fetchall()

    for row in rows:
        print(row)


In [16]:
# create a database connection
conn = create_connection(db_file)
with conn:
    print("1. Query task by priority:")
    select_task_by_priority(conn, 1)

    print("2. Query all tasks")
    select_all_tasks(conn)

1. Query task by priority:
(1, 'Analyze the requirements of the app', 1, 1, 1, '2015-01-01', '2015-01-02')
(2, 'Confirm with user about the top requirements', 1, 1, 1, '2015-01-03', '2015-01-05')
2. Query all tasks
(1, 'Analyze the requirements of the app', 1, 1, 1, '2015-01-01', '2015-01-02')
(2, 'Confirm with user about the top requirements', 1, 1, 1, '2015-01-03', '2015-01-05')


In [17]:
! ls -l "${HOME}/bdl03-1/sqlite"

total 12
-rw-r--r-- 1 grossnik grossnik 12288 Feb 15 18:44 pythonsqlite.db
